In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, models
from transformers import ViTFeatureExtractor, ViTForImageClassification, TrainingArguments, Trainer
import timm

In [2]:
from dataprocessing import DataLoaderWrapper

data=DataLoaderWrapper(batch_size=32, image_size=224)
celebA_train,celebA_val,celebA_test=data.initialize_celebA_dataloaders()
adience=data.initialize_adience_dataloaders()

\

Opening dataset in read-only mode as you don't have write permissions.


-

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/adience



/

...

\

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/celeb-a-test



/

hub://activeloop/celeb-a-test loaded successfully.



/Users/tulipmajumder/opt/anaconda3/lib/python3.9/site-packages/deeplake/integrations/pytorch/common.py:126: UserWarning: Decode method for tensors ['images'] is defaulting to numpy. Please consider specifying a decode_method in .pytorch() that maximizes the data preprocessing speed based on your transformation.
  warnings.warn(


In [ ]:
model = timm.create_model('vit_base_patch16_224', pretrained=True)

model.eval()  # Set the model to evaluation mode

# Prepare to collect the predictions
all_probabilities = []

with torch.no_grad():
    for test_images, age_labels, gender_labels in celebA_val:
        out = model(test_images)
        
        # Apply softmax to calculate probabilities
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        
        # Collect the probabilities for further processing or analysis
        all_probabilities.append(probabilities)
        
#probabilities = torch.nn.functional.softmax(out[0], dim=0)